# Packages 

In [5]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from math import *
from sklearn.metrics import r2_score
import seaborn as sns
import warnings
warnings.filterwarnings("ignore") 
%matplotlib inline
from tqdm import tqdm
import csv as csv

In [6]:
output_path = '/Users/lanauaa/Desktop/Viral_dynamics_project/Data_replication_immunotron/'
experiment_name = '20230306_MHV_HeLa_kinetics'
name_rawdata_file = '20230202_MHV_HeLa_sybr_plate'
plate_number='3'

# Import Data 

In [21]:
#Convert txt to csv
convertion_to_csv = pd.read_csv('{}/{}/Data/{}{}.txt'.format(output_path,experiment_name,name_rawdata_file,plate_number),sep='\t')
convertion_to_csv.drop(['Include','Name','Concentration','Standard','Status'], axis=1,inplace=True)
convertion_to_csv.to_csv('{}/{}/Data/{}{}.csv'.format(output_path,experiment_name,name_rawdata_file,plate_number))

#Import data
PCR_data_raw = pd.read_csv('{}/{}/Data/{}{}.csv'.format(output_path,experiment_name,name_rawdata_file,plate_number)).drop(['Unnamed: 0'],axis=1).rename(columns={'Pos':'Tag','Cp':'CT'})
                        
PCR_data = PCR_data_raw.dropna(axis=0, inplace=False)

#Define Titers and Timepoints
titers_list=[10,1,0.1,0.01,0.001,0.0001,0.00001,0]
timepoints_list=[24,20,16,12,8,5,4,3,2,1.5,1,0.5]

PCR_data

     Include  Color  Pos        Name     Cp  Concentration  Standard  \
0       True    255   A1    Sample 1  16.02            NaN         0   
1       True    255   A2    Sample 2  32.79            NaN         0   
2       True    255   A3    Sample 3  15.05            NaN         0   
3       True    255   A4    Sample 4  25.83            NaN         0   
4       True    255   A5    Sample 5  27.35            NaN         0   
..       ...    ...  ...         ...    ...            ...       ...   
379     True    255  P20  Sample 380  36.86            NaN         0   
380     True  65280  P21  Sample 381    NaN            NaN         0   
381     True    255  P22  Sample 382  36.74            NaN         0   
382     True    255  P23  Sample 383  33.49            NaN         0   
383     True    255  P24  Sample 384  40.00            NaN         0   

                                                Status  
0                                                  NaN  
1                    

,Color,Tag,CT
0,255,A1,18.27
1,255,A2,17.50
2,255,A3,15.77
3,255,A4,20.17
4,255,A5,16.15
...,...,...,...
379,255,P20,40.00
380,255,P21,26.84
381,255,P22,27.98
382,255,P23,34.90


In [13]:
#Convert txt to csv
convertion_to_csv = pd.read_csv('{}/{}/Data/{}{}.txt'.format(output_path,experiment_name,name_rawdata_file,plate_number),sep='\t')
convertion_to_csv.drop(['Include','Name','Concentration','Standard','Status'], axis=1,inplace=True)
convertion_to_csv.to_csv('{}/{}/Data/{}{}.csv'.format(output_path,experiment_name,name_rawdata_file,plate_number))

#Import data
PCR_data_raw = pd.read_csv('{}/{}/Data/{}{}.csv'.format(output_path,experiment_name,name_rawdata_file,plate_number)).drop(['Unnamed: 0'],axis=1).rename(columns={'Pos':'Tag','Cp':'CT'})
PCR_data = PCR_data_raw.dropna(axis=0, inplace=False)    


# #define HK and GOI
# PCR_data['GeneType']=['HK' if int(x[1:])<13 else 'GOI' for x in PCR_data['Tag']]
# PCR_data = PCR_data.set_index(['GeneType'],append=True)

# sub_df=PCR_data.query('GeneType=="HK"').reset_index().drop(['level_0'],axis=1)
# sub_df['HK']=sub_df['CT']
# sub_2df=PCR_data.query('GeneType=="GOI"').reset_index().drop(['level_0'],axis=1)
# sub_df['GOI']=sub_2df['CT']
# PCR_data=sub_df.drop(['CT','GeneType'],axis=1)

# #set timepoints and titers
# df_list=[]

# PCR_data['numbers']=[int(x[1:]) for x in PCR_data['Tag']]
# numbers_list=PCR_data['numbers'].unique()
# for i,time in enumerate(timepoints_list):
#     i=i+1
#     sub_df = PCR_data.query('numbers==@i')
#     sub_df['Timepoint']=time
#     df_list.append(sub_df)
# PCR_data=pd.concat(df_list)

# df2_list=[]
# PCR_data['letters']=[x[0] for x in PCR_data['Tag']]
# letters_list=PCR_data['letters'].unique()
# for i,titer in enumerate(titers_list):
#     j=letters_list[0+2*i]
#     k=letters_list[1+2*i]
#     sub_df = PCR_data.query('letters==@j')
#     sub_df['Titer']= titer
#     df2_list.append(sub_df)
#     sub_df = PCR_data.query('letters==@k')
#     sub_df['Titer']= titer
#     df2_list.append(sub_df)

# PCR_data=pd.concat(df2_list)


#delta2CT method
#mean replicate 
timepoints_list=PCR_data.Timepoint.unique()
titers_list=PCR_data.Titer.unique()
mean_rep_list=[]
for titer in titers_list:
    for time in timepoints_list:
        sub_rep1_df = PCR_data.query('Titer==@titer and Timepoint==@timepoint and Replicate=="Rep1"')
        sub_rep2_df = PCR_data.query('Titer==@titer and Timepoint==@timepoint and Replicate=="Rep2"')
        sub_rep1_df['CT'] = np.mean(sub_rep1_df['CT'],sub_rep2_df['CT'])
        mean_rep_list.append(sub_rep1_df)

PCR_data=pd.concat(mean_rep_list).drop(['Replicate'],axis=1)

#DeltaCT
PCR_data.unstack()
GOI_list=PCR_data.Gene.unique().drop('HK',axis=0)
DeltaCT_list=[]
for goi in GOI_list:
    DeltaCT_df=PCR_data.drop('HK',axis=0)
    DeltaCT_df['DeltaCT']=PCR_data.query('Gene=@goi')['CT']-PCR_data.query('Gene=="HK"')['CT']
    DeltaCT_list.append(DeltaCT_df)
DeltaCT_df=pd.concat(DeltaCT_list)

#

# pcr_list=[]
# for titer in titers_list:
#     sub_df=PCR_data.query('Titer==@titer')
#     sub_pcr_analysed = pd.DataFrame({'Timepoint':timepoints_list, 'Titer':titer},index=[0:len(timepoints_list)])
#     sub_pcr_analysed['DeltaCT']=sub_df.query('GeneType=="HK"')['CT']-sub_df.query('GeneType=="GOI"')['CT']
#     pcr_list.append(sub_pcr_analysed)
# sub_pcr_analysed = pd.concat(pcr_list)
    
# sub_df=sub_pcr_analysed.query('Timepoint==0')
# min_deltaCT_list = np.mean(sub_df.query(rep

# #delta2CT method
# df3_list=[]
# list_1=[]
# PCR_data['DeltaCT']=PCR_data['GOI']-PCR_data['HK']
# PCR_data.dropna(inplace=True)
# print(PCR_data)
# list_1=PCR_data.query('letters=="O" or letters=="P"')
# min_value = min(list_1['DeltaCT'])
# min_time=min(list_1.Timepoint.unique())
# print(min_time)
# min_df=list_1.query('Timepoint==@min_time')
# print(min_df)
# delta= np.mean(min_df['DeltaCT'])

# for number in numbers_list:
#     sub_df=PCR_data.query('numbers==@number')
#     sub_df['Delta2CT']=sub_df['DeltaCT']-delta
#     df3_list.append(sub_df)
# PCR_data=pd.concat(df3_list).drop(['numbers','letters'],axis=1)
# PCR_data["GeneExpression"]=2**(-PCR_data['Delta2CT'])

# #Mean of technical duplicate/triplicate 
# list4_df=[]
# for time in timepoints_list:
#     sub_df=PCR_data.query('Timepoint==@time')
#     for titer in titers_list:
#         sub_sub_df = sub_df.query('Titer==@titer')
#         mean_expression = np.mean(sub_sub_df['GeneExpression'])
#         sub_sub_df['GeneExpression']=mean_expression
#         mean_df=sub_sub_df.drop_duplicates('GeneExpression').drop(['Tag','HK','GOI','DeltaCT','Delta2CT','Color'],axis=1)
#         list4_df.append(mean_df)
#     PCR_data_mean = pd.concat(list4_df)

PCR_data_mean.sort_values(['Titer','Timepoint'],inplace=True)
PCR_data_mean.to_csv('{}/{}/Data/Analyzed_PCR_Data_Plate_{}.csv'.format(output_path,experiment_name,plate_number))

PCR_data_mean

   Timepoint  Titer
0        NaN    NaN
     Color  Tag     HK    GOI  numbers  Timepoint letters  Titer  DeltaCT
0      255   A1  18.27  13.69        1       24.0       A   10.0    -4.58
1      255   A2  17.50  14.76        2       20.0       A   10.0    -2.74
2      255   A3  15.77  15.57        3       16.0       A   10.0    -0.20
3      255   A4  20.17  24.25        4       12.0       A   10.0     4.08
4      255   A5  16.15  21.04        5        8.0       A   10.0     4.89
..     ...  ...    ...    ...      ...        ...     ...    ...      ...
175    255   P7  15.64  27.61        7        4.0       P    0.0    11.97
176    255   P8  28.73  33.72        8        3.0       P    0.0     4.99
177    255  P10  17.15  29.80       10        1.5       P    0.0    12.65
178    255  P11  19.24  30.54       11        1.0       P    0.0    11.30
179    255  P12  15.76  29.85       12        0.5       P    0.0    14.09

[180 rows x 9 columns]
0.5
     Color  Tag     HK    GOI  numbers  Time

,Timepoint,Titer,GeneExpression
168,0.5,0.0,2.796283
167,1.0,0.0,1.127509
166,1.5,0.0,0.439677
165,2.0,0.0,5.483131
164,3.0,0.0,50.750317
...,...,...,...
4,8.0,10.0,71.233617
3,12.0,10.0,1081.036067
2,16.0,10.0,4212.870603
1,20.0,10.0,22724.305752
